### Imports

In [83]:
import os
import requests
import pdfplumber
import re
import pandas as pd

### Paths

In [107]:
CWD = os.path.join(os.getcwd())
SOURCE_PATH = os.path.join(CWD, 'csv', 'gesetze.csv')
TARGET_PATH = os.path.join(CWD, 'csv', 'gesetze_mit_drucksachen.csv')
DRUCKSACHE_PATH = os.path.join(CWD, 'csv', 'drucksachen_details.csv')
DRUCKSACHE_WITH_DESC_PATH = os.path.join(CWD, 'csv', 'drucksachen_details_with_desc.csv')
API_KEY = 'I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21'

### Load data

In [85]:
df = pd.read_csv(SOURCE_PATH)
df.head()

,Unnamed: 0,date,title,id,topic,url,drucksache
0,0,2024-12-19,Gesetzentwurf zur Änderung von Artikel 93 und ...,939,Geschäftsordnung,/parlament/plenum/abstimmung/abstimmung?id=939,['https://dserver.bundestag.de/btd/20/129/2012...
1,1,2024-12-16,Antrag gemäß Artikel 68 des Grundgesetzes (Ver...,938,Parlament,/parlament/plenum/abstimmung/abstimmung?id=938,['https://dserver.bundestag.de/btd/20/141/2014...
2,2,2024-10-18,Änderungsantrag zur „Verbesserung der inneren ...,929,Inneres,/parlament/plenum/abstimmung/abstimmung?id=929,['https://dserver.bundestag.de/btd/20/128/2012...
3,3,2024-10-18,Artikel 5 des Entwurfs „Verbesserung der inner...,930,Inneres,/parlament/plenum/abstimmung/abstimmung?id=930,['https://dserver.bundestag.de/btd/20/128/2012...
4,4,2024-10-18,Gesetzentwurf „Verbesserung der Terrorismusbek...,931,Inneres,/parlament/plenum/abstimmung/abstimmung?id=931,['https://dserver.bundestag.de/btd/20/128/2012...


### API Key

In [86]:
API_KEY = 'I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21' 

### Create Drucksache Details CSV

In [87]:
def extract_document_number(url):
    parts = url.split('/')
    wahlperiode = parts[4]  # nach "btd/"
    pdf_name = parts[-1].replace('.pdf', '')
    dokumentnummer = pdf_name[len(wahlperiode):]
    return f"{wahlperiode}/{dokumentnummer}"

In [88]:
def get_document_info(doc_numbers):
    api_url = f"https://search.dip.bundestag.de/api/v1/drucksache-text?f.dokumentnummer={doc_numbers}&format=json&apikey={API_KEY}"
    response = requests.get(api_url, headers={'accept': 'application/json'})
    if response.status_code == 200:
        data = response.json()
        if data["numFound"] > 0:
            documents = data["documents"]
            doc_info = {}
            for doc in documents:
                doc_nr = doc["dokumentnummer"]
                doc_id = doc["id"]
                doc_type = doc.get("drucksachetyp", None)
                doc_text = doc.get("text", None)
                doc_info[doc_nr] = {"id": doc_id, "type": doc_type, "text": doc_text}
            return doc_info
    return {}

In [89]:
def process_csv(input_file, output_file, drucksache_output, num_samples=None):
    df = pd.read_csv(input_file)
    if num_samples is not None:
        df = df.head(num_samples)
    df['drucksache_referenz'] = None
    drucksache_data = []
    for i, row in df.iterrows():
        try:
            urls = eval(row['drucksache'])
            referenzen = []
            doc_numbers = [extract_document_number(url) for url in urls]
            doc_info = get_document_info('&f.dokumentnummer='.join(doc_numbers))
            for url, doc_number in zip(urls, doc_numbers):
                doc_data = doc_info.get(doc_number, {})
                drucksache_data.append({
                    "referenznummer": doc_data.get("id", None),
                    "docNr": doc_number,
                    "docTyp": doc_data.get("type", None),
                    "docText": doc_data.get("text", None)
                })
                referenzen.append(doc_data.get("id", None))
            df.at[i, 'drucksache_referenz'] = str(referenzen)
        except Exception:
            continue
    df.to_csv(output_file, index=False)
    drucksache_df = pd.DataFrame(drucksache_data)
    drucksache_df.to_csv(drucksache_output, index=False)

### Add DocType & DocNr

In [90]:
url = 'https://dserver.bundestag.de/btd/20/143/2014302.pdf'
docNr = extract_document_number(url)
print(docNr)

doc_info = get_document_info(docNr)
print(doc_info)

20/14302
{'20/14302': {'id': '277864', 'type': 'Beschlussempfehlung und Bericht', 'text': 'Deutscher Bundestag Drucksache 20/14302 \n20. Wahlperiode 18.12.2024 \nBeschlussempfehlung und Bericht \ndes Rechtsausschusses (6. Ausschuss) \na) zu dem Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜNDNIS 90/DIE\nGRÜNEN und FDP sowie des Abgeordneten Stefan Seidler\n– Drucksache 20/12977 –\nEntwurf eines Gesetzes zur Änderung des Grundgesetzes \n(Artikel 93 und 94) \nb) zu dem Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜNDNIS 90/DIE\nGRÜNEN und FDP sowie des Abgeordneten Stefan Seidler\n– Drucksache 20/12978 –\nEntwurf eines Gesetzes zur Änderung des  \nBundesverfassungsgerichtsgesetzes und des \nUntersuchungsausschussgesetzes \nA. Problem\nZu Buchstabe a)\nDie den Gesetzentwurf vorlegenden Fraktionen und der Abgeordnete Stefan\nSeidler weisen darauf hin, dass sich das Grundgesetz (GG) für eine starke\nVerfassungsgerichtsbarkeit entschieden habe. Diese zentrale Weichenstellung lasse sich \nan de

In [91]:
process_csv(SOURCE_PATH, TARGET_PATH, DRUCKSACHE_PATH, num_samples=5) # None


### Extract text for summary

In [108]:
def extractGesetzentwurf(doc_text):
    # Filter
    problem_section = re.search(r'A\. Problem und Ziel(.*?)(?=B\. Lösung|$)', doc_text, re.DOTALL)
    loesung_section = re.search(r'B\. Lösung(.*?)(?=C\. Alternativen|$)', doc_text, re.DOTALL)
    
    # Extracted texts
    if problem_section and loesung_section:
        problem_text = problem_section.group(1).strip()
        loesung_text = loesung_section.group(1).strip()
        
        # Combine with [SEP] 
        extracted_text = f"{problem_text} [SEP] {loesung_text}"
        return extracted_text
    else:
        return "Sections not found"

In [109]:
def extractForTyp2():
    print("Extracting information for Typ2...")
    return "Typ2 extraction complete."

def extractForTyp3():
    print("Extracting information for Typ3...")
    return "Typ3 extraction complete."

def extractForOtherTypes():
    print("Extracting information for other document types...")
    return "Other types extraction complete."

In [110]:
def process_drucksachen_details(csv_path, output_path, samples=None):
    df = pd.read_csv(csv_path)
    
    if samples is not None:
        df = df.head(samples)
    
    df['extractedText'] = None
    
    for i, row in df.iterrows():
        doc_type = row['docTyp']
        doc_text = row['docText']
        
        if doc_type == "Gesetzentwurf":
            df.at[i, 'extractedText'] = extractGesetzentwurf(doc_text)
        elif doc_type == "Typ2":
            df.at[i, 'extractedText'] = extractForTyp2()
        elif doc_type == "Typ3":
            df.at[i, 'extractedText'] = extractForTyp3()
        else:
            df.at[i, 'extractedText'] = extractForOtherTypes()
    
    df.to_csv(output_path, index=False)


### Test extraction for Gesetzentwurf

In [112]:
referenznummer = 276076

In [113]:
df = pd.read_csv(DRUCKSACHE_PATH)
row = df[df['referenznummer'] == referenznummer].iloc[0]
doc_text = row['docText']

In [114]:
extracted_text = extractGesetzentwurf(doc_text)
print(extracted_text)

Das Grundgesetz hat sich für eine starke Verfassungsgerichtsbarkeit entschieden. 
Diese zentrale Weichenstellung lässt sich an den – auch im internationalen
Vergleich – weitreichenden Entscheidungsbefugnissen ablesen, die das Grundgesetz 
dem Bundesverfassungsgericht zuweist. Auf dieser Grundlage hat sich das
Bundesverfassungsgericht als Garant der freiheitlich-demokratischen Ordnung und 
als für Staat und Gesellschaft wesentliches Verfassungsorgan mittlerweile fest 
etabliert. 
Als das Grundgesetz am 24. Mai 1949 in Kraft trat, war die neuartige Institution 
„Bundesverfassungsgericht“ verfassungsrechtlich nur teilweise näher ausgeformt. 
Auch das Verständnis des Gerichts als Verfassungsorgan war anfangs noch nicht 
allgemein konsentiert. Es oblag in der Folge sehr weitgehend dem einfachen
Gesetzgeber, Stellung und Struktur des Gerichts näher zu regeln. Dieser Aufgabe ist 
der Gesetzgeber mit dem Gesetz über das Bundesverfassungsgericht
(Bundesverfassungsgerichtsgesetz – BVerfGG) nachg

### Extract all

In [111]:
process_drucksachen_details(DRUCKSACHE_PATH, DRUCKSACHE_WITH_DESC_PATH, samples=5)

Extracting information for other document types...
Extracting information for other document types...
Extracting information for other document types...
